# Prep SLURM commands

## Define parameters

In [12]:
# Constant Arguments Setup
constant_args_df <- data.frame(
  outdir = "./output/",
  chunk1 = 1000000,
  chunk2 = 1000010,
  #snp_data_path = "/dcs04/lieber/statsgen/mnagle/mwas/gwas/libd_chr1.pgen",
  #methylation_data_path = "/dcs04/lieber/statsgen/mnagle/mwas/pheno/dlpfc/out/chr1_AA.rda",
  snp_data_path = "/Users/michaelnagle/code/mwas/gwas/libd_chr1.pgen",
  methylation_data_path = "/Users/michaelnagle/code/mwas/pheno/dlpfc/out/chr1_AA.rda",  
  verbose = TRUE,
  lambda_choice = "1se",
  alphas = "0.25,0.5,0.75,1",
  #num_cores = "all",
  allow_inefficient_parallelization = FALSE,
  n_fold = 5,
  window_sizes = "1000,2000,5000,10000,20000,50000,100000,500000",
  #tag = format(Sys.time(), "%Y%m%d-%H%M%S"),
  save_evaluation_results_each_fold = FALSE,
  save_glmnet_object = FALSE,
  cv_eval_mode = "dynamic"
)

# Convert alphas to a comma-separated string
#constant_args_df$alphas <- sapply(constant_args_df$alphas, function(x) paste(x, collapse = ","))


# Varying parameters
cv_eval_modes <- c("static", "dynamic")
cores_per_alphas <- c("all", "1")

## Generate and deploy SLURM calls

In [13]:
# Function to generate SLURM script
generate_slurm_script <- function(args, tag) {
  mem_allocation <- if (args$cores_per_alpha == "all") "0" else "16G"
  args_string <- paste("--", names(args), "=", args, sep = "", collapse = " ")
  args_string <- paste(args_string, " --tag=", tag, sep = "")  # Append tag to the argument string

  slurm_script <- paste(
    "#!/bin/bash\n",
    if (args$cores_per_alpha == "all") "#SBATCH --exclusive\n" else "#SBATCH --cpus-per-task=1\n",
    "#SBATCH --mem=", mem_allocation, "\n",
    "#SBATCH --output=slurm_output_", tag, ".out\n",
    "#SBATCH --job-name=", tag, "\n",
    "Rscript scripts/CLI.R ", args_string, "\n",
    sep = ""
  )
  return(slurm_script)
}

# Varying parameters
cv_eval_modes <- c("static", "dynamic")
cores_per_alphas <- c("all", "1")

# Loop through each combination
for (cv_eval_mode in cv_eval_modes) {
  for (cores_per_alpha in cores_per_alphas) {
    # Update constant_args_df for the current combination
    constant_args_df$cv_eval_mode <- cv_eval_mode
    constant_args_df$cores_per_alpha <- cores_per_alpha

    # Generate tag
    snp_base <- tools::file_path_sans_ext(basename(constant_args_df$snp_data_path))
    meth_base <- tools::file_path_sans_ext(basename(constant_args_df$methylation_data_path))
    datetime_str <- format(Sys.time(), "%Y%m%d-%H%M%S")
    tag <- paste(snp_base, meth_base, cv_eval_mode, paste0(cores_per_alpha, "core"), datetime_str, sep = "-")

    # Generate and print SLURM script
    slurm_script <- generate_slurm_script(constant_args_df, tag)
    cat(slurm_script, "\n\n")

    # Additional logic to submit jobs with sbatch or srun, including job submission limits and intervals
    # ...
  }
}


#!/bin/bash
#SBATCH --exclusive
#SBATCH --mem=0
#SBATCH --output=slurm_output_libd_chr1-chr1_AA-static-allcore-20240129-123107.out
#SBATCH --job-name=libd_chr1-chr1_AA-static-allcore-20240129-123107
Rscript scripts/CLI.R --outdir=./output/ --chunk1=1e+06 --chunk2=1000010 --snp_data_path=/Users/michaelnagle/code/mwas/gwas/libd_chr1.pgen --methylation_data_path=/Users/michaelnagle/code/mwas/pheno/dlpfc/out/chr1_AA.rda --verbose=TRUE --lambda_choice=1se --alphas=0.25,0.5,0,75,1 --allow_inefficient_parallelization=FALSE --n_fold=5 --window_sizes=1000,2000,5000,10000,20000,50000,100000,500000 --save_evaluation_results_each_fold=FALSE --save_glmnet_object=FALSE --cv_eval_mode=static --cores_per_alpha=all --tag=libd_chr1-chr1_AA-static-allcore-20240129-123107
 

#!/bin/bash
#SBATCH --cpus-per-task=1
#SBATCH --mem=16G
#SBATCH --output=slurm_output_libd_chr1-chr1_AA-static-1core-20240129-123107.out
#SBATCH --job-name=libd_chr1-chr1_AA-static-1core-20240129-123107
Rscript scripts/CLI.R --outdir=